In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pymoo.core.problem import Problem
import aerosandbox as asb
import aerosandbox.numpy as np
import matplotlib.pyplot as plt
import aerosandbox.tools.pretty_plots as p

In [3]:
import os
import sys
sys.path.append(os.path.abspath("") + "../../../")
from nnaero.optimization.design_variables import DesignVariables

In [4]:
# We aren't just optimizing for a single point but for multiple points
CL_multipoint_targets = np.array([0.8, 1.0, 1.2, 1.4, 1.5, 1.6])
CL_multipoint_weights = np.array([5, 6, 7, 8, 9, 10])


Re = 500e3 * (CL_multipoint_targets / 1.25) ** -0.5
mach = 0.03

initial_guess_airfoil = asb.KulfanAirfoil("naca0012")
initial_guess_airfoil.name = "Initial Guess (NACA0012)"

opti = asb.Opti()

optimized_airfoil = asb.KulfanAirfoil(
    name="Optimized",
    lower_weights=opti.variable(
        init_guess=initial_guess_airfoil.lower_weights,
        lower_bound=-0.5,
        upper_bound=0.25,
    ),
    upper_weights=opti.variable(
        init_guess=initial_guess_airfoil.upper_weights,
        lower_bound=-0.25,
        upper_bound=0.5,
    ),
    leading_edge_weight=opti.variable(
        init_guess=initial_guess_airfoil.leading_edge_weight,
        lower_bound=-1,
        upper_bound=1,
    ),
    TE_thickness=0,
)

In [10]:
alpha = opti.variable(
    init_guess=np.degrees(CL_multipoint_targets / (2 * np.pi)),
    lower_bound=-5,
    upper_bound=18,
)

aero = optimized_airfoil.get_aero_from_neuralfoil(
    alpha=alpha,
    Re=Re,
    mach=mach,
)

opti.subject_to(
    [
        aero["analysis_confidence"] > 0.90,
        aero["CL"] == CL_multipoint_targets,
        np.diff(alpha) > 0,
        aero["CM"] >= -0.133,
        optimized_airfoil.local_thickness(x_over_c=0.33) >= 0.128,
        optimized_airfoil.local_thickness(x_over_c=0.90) >= 0.014,
        optimized_airfoil.TE_angle()
        >= 6.03,  # Modified from Drela's 6.25 to match DAE-11 case
        optimized_airfoil.lower_weights[0] < -0.05,
        optimized_airfoil.upper_weights[0] > 0.05,
        optimized_airfoil.local_thickness() > 0,
    ]
)

get_wiggliness = lambda af: sum(
    [
        np.sum(np.diff(np.diff(array)) ** 2)
        for array in [af.lower_weights, af.upper_weights]
    ]
)

opti.subject_to(
    get_wiggliness(optimized_airfoil) < 2 * get_wiggliness(initial_guess_airfoil)
)

opti.minimize(np.mean(aero["CD"] * CL_multipoint_weights))

sol = opti.solve(
    behavior_on_failure="return_last",
    options={"ipopt.mu_strategy": "monotone", "ipopt.start_with_resto": "yes"},
)

optimized_airfoil = sol(optimized_airfoil)
aero = sol(aero)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      108
Number of nonzeros in inequality constraint Jacobian.:     2043
Number of nonzeros in Lagrangian Hessian.............:      261

Total number of variables............................:       23
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:      170
        inequality constraints with only lower bounds:      145
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       25

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.1395771e-01 4.43e-01 1.07e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [11]:
aero["CL"]

array([0.8, 1. , 1.2, 1.4, 1.5, 1.6])

In [5]:
variables = DesignVariables()
variables.add([
    ("lower_weights", initial_guess_airfoil.lower_weights, -0.5,0.25),
    ("upper_weights", initial_guess_airfoil.upper_weights, -0.25,0.5),
    ("leading_edge_weight", initial_guess_airfoil.leading_edge_weight, -1,1),
])

variables.add("alpha", np.degrees(CL_multipoint_targets / (2 * np.pi)), lower_bound=-5, upper_bound=18)

In [6]:
variables.get_bounds_vectors()

(array([-0.5 , -0.5 , -0.5 , -0.5 , -0.5 , -0.5 , -0.5 , -0.5 , -0.25,
        -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -1.  , -5.  ,
        -5.  , -5.  , -5.  , -5.  , -5.  ]),
 array([ 0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.5 ,
         0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ,  1.  , 18.  ,
        18.  , 18.  , 18.  , 18.  , 18.  ]))

In [9]:
xl,xu  = variables.get_bounds_vectors()

In [ ]:
class MyProblem(Problem):
    def __init__(self, variables:DesignVariables):
        n_var = variables._total_dof
        xl,xu  = variables.get_bounds_vectors()
        super().__init__(n_var=n_var, 
                         n_obj=1,
                         xl=xl,
                         xu=xu,
                         n_ieq_constr=124, n_eq_constr=6,)
        
    def evaluate_airfoil(self,variables):
        
        # Also, include upper and lower bounds while making up variables. 
        
        optimized_airfoil = asb.KulfanAirfoil(
            name="Optimized",
            lower_weights=variables.lower_weights,
            upper_weights=variables.upper_weights,
            leading_edge_weight=variables.leading_edge_weight,
            TE_thickness=0,
        )
        
        # this part can be implemented with individual airfoils also, but as we are calling a neural network itself we can make this also in batch
        aero = optimized_airfoil.get_aero_from_neuralfoil(
            alpha=alpha,
            Re=Re,
            mach=mach,
        )
        
        get_wiggliness = lambda af: sum(
            [
                np.sum(np.diff(np.diff(array)) ** 2)
                for array in [af.lower_weights, af.upper_weights]
            ]
        )
        
        # I think on the basis of this I should put super.__init__() of pymoo
        # It's not possible to stack up all this constraints in a single column, 
        ineq_constraints = [
            0.9 - aero["analysis_confidence"],
            -np.diff(variables.alpha),
            -0.133-aero["CM"],
            -optimized_airfoil.local_thickness(x_over_c=0.33) + 0.128,
            -optimized_airfoil.local_thickness(x_over_c=0.90) + 0.014,
            -optimized_airfoil.TE_angle() + 6.03,
            optimized_airfoil.lower_weights[0] + 0.05,
            -optimized_airfoil.upper_weights[0] + 0.05,
            -optimized_airfoil.local_thickness(),
            get_wiggliness(optimized_airfoil) - 2 * get_wiggliness(initial_guess_airfoil)
        ]

        ineq_constraints = np.concatenate([np.atleast_1d(v) for v in ineq_constraints])[:, None]
        ineq_constraints.shape
        
        eq_constraints = np.column_stack([
            aero["CL"] - CL_multipoint_targets
        ])
        
        objective = np.mean(aero["CD"] * CL_multipoint_weights)
        
        return objective, eq_constraints, ineq_constraints
        
    def _evaluate(self, x, out, *args, **kwargs):
        
        pop_size = x.shape[0]
        pop_objective = np.empty((pop_size, 1))
        pop_eq_constraint = np.empty((pop_size, 6))
        pop_ineq_constraint = np.empty((pop_size, 124))
        
        for i in range(pop_size):
            objective, eq_constraints, ineq_constraints = self.evaluate_airfoil(variables)
            pop_objective[i]=  objective
            pop_eq_constraint[i]=  eq_constraints
            pop_ineq_constraint[i]=  ineq_constraints
            
        out["F"] = pop_objective
        out["G"] = pop_ineq_constraint
        out["H"] = pop_eq_constraint

In [32]:
prob = MyProblem(variables)
prob.n_obj

1

In [36]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
algorithm = NSGA2(pop_size=100)

In [37]:
res = minimize(prob,
               algorithm,
               seed=1)

print(res.algorithm.n_gen)

Exception: Implicit conversion of symbolic CasADi type to numeric matrix not supported.
This may occur when you pass a CasADi object to a numpy function.
Use an equivalent CasADi function instead of that numpy function.

In [28]:
ineq_constraints = [
            0.9 - aero["analysis_confidence"],
            -np.diff(variables.alpha),
            -0.133-aero["CM"],
            -optimized_airfoil.local_thickness(x_over_c=0.33) + 0.128,
            -optimized_airfoil.local_thickness(x_over_c=0.90) + 0.014,
            -optimized_airfoil.TE_angle() + 6.03,
            optimized_airfoil.lower_weights[0] + 0.05,
            -optimized_airfoil.upper_weights[0] + 0.05,
            -optimized_airfoil.local_thickness(),
            get_wiggliness(optimized_airfoil) - 2 * get_wiggliness(initial_guess_airfoil)
        ]

ineq_constraints = np.concatenate([np.atleast_1d(v) for v in ineq_constraints])[:, None]
ineq_constraints.shape

(124, 1)

In [ ]:
# class MyProblem(Problem):
#     def __init__(self):
#         super().__init__(n_var=2,
#                          n_obj=2,
#                          xl=-2.0,
#                          xu=2.0)

#     def _evaluate(self, x, out, *args, **kwargs):
#         f1 = 100 * (x[:, 0]**2 + x[:, 1]**2)
#         f2 = (x[:, 0]-1)**2 + x[:, 1]**2
#         out["F"] = np.column_stack([f1, f2])
#         out["G"] = np.column_stack([f1, f2])

## Basic Pymoo implementation

In [ ]:
import numpy as np

from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination

In [32]:

class MyVectorizedProblem(Problem):

    def __init__(self):
        super().__init__(
            n_var=2,        # number of decision variables
            n_obj=2,        # number of objectives
            n_constr=2,     # number of constraints
            xl=np.array([0.0, 0.0]),
            xu=np.array([2.0, 2.0]),
            elementwise=False  # <-- IMPORTANT: vectorized evaluation
        )

    def _evaluate(self, X, out, *args, **kwargs):
        """
        X shape: (n_individuals, n_var)
        """

        x1 = X[:, 0]
        x2 = X[:, 1]

        # Objectives
        f1 = x1**2 + x2**2
        f2 = (x1 - 1)**2 + (x2 - 1)**2

        out["F"] = np.column_stack([f1, f2])

        # Constraints (must be <= 0)
        g1 = x1 + x2 - 1.0
        g2 = 0.5 - x1

        out["G"] = np.column_stack([g1, g2])


# Algorithm
algorithm = NSGA2(
    pop_size=100
)

# Termination criterion
termination = get_termination("n_gen", 100)

# Optimization
res = minimize(
    MyVectorizedProblem(),
    algorithm,
    termination,
    seed=1,
    save_history=False,
    verbose=False
)

# Results
print("Pareto-optimal solutions (X):")
print(res.X)

print("\nObjective values (F):")
print(res.F)


Pareto-optimal solutions (X):
[[0.50020052 0.00259388]
 [0.50093304 0.49904846]
 [0.50036689 0.11281126]
 [0.50022639 0.12908919]
 [0.50030836 0.20149919]
 [0.50031138 0.16688373]
 [0.50021037 0.07347205]
 [0.50022527 0.14689112]
 [0.50032901 0.08415559]
 [0.50021617 0.24493306]
 [0.50021742 0.26038886]
 [0.50039043 0.24058023]
 [0.50093304 0.49813432]
 [0.50040808 0.48995138]
 [0.50019528 0.36352292]
 [0.50659031 0.4581981 ]
 [0.50023914 0.13738711]
 [0.50030033 0.32868707]
 [0.50038956 0.47708036]
 [0.50020229 0.39482088]
 [0.50022527 0.1559232 ]
 [0.50026054 0.21319407]
 [0.50022142 0.44422121]
 [0.50040426 0.34986276]
 [0.50022598 0.42189279]
 [0.50624588 0.28146558]
 [0.500387   0.19601634]
 [0.50049964 0.43849787]
 [0.50029271 0.23099573]
 [0.50040067 0.27894371]
 [0.50039851 0.41461928]
 [0.50020213 0.25470871]
 [0.50029571 0.35860148]
 [0.50029755 0.35478227]
 [0.50123384 0.40713046]
 [0.50022614 0.44855274]
 [0.50173503 0.45609549]
 [0.50030767 0.17775999]
 [0.50029798 0.40672